# 📦 Inventory Optimization Analysis

### 🧠 Objective
To analyze raw material procurement data, identify overstocked and underutilized items, and recommend actions to reduce inventory holding costs.

---

## 1. 🧹 Import Libraries & Load Data

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# load data set
df = pd.read_csv("./data/full_data/data.csv")

C:\Users\soora\AppData\Local\Temp\ipykernel_16872\1588343676.py:8: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data/full_data/data.csv")


## 2. 🧼 Data Cleaning
- Check for missing values
- Check for duplicate entries
- Ensure correct data types

In [64]:
# df.info()
# df.isnull().sum()

# Drop duplicates
print("duplicate rows : ", df.duplicated().sum())
df.drop_duplicates(inplace=True)

df.columns

duplicate rows :  0


Index(['Material', 'Material Description', 'Plant', 'Storage Location',
       'Movement Type', 'Posting Date', 'Qty in Un. of Entry', 'Unit of Entry',
       'Batch', 'Order', 'Movement Type Text', 'Document Date', 'Qty in OPUn',
       'Order Price Unit', 'Order Unit', 'Qty in order unit', 'Entry Date',
       'Time of Entry', 'Amount in LC', 'Purchase Order', 'Item',
       'Cost Center', 'Movement indicator', 'Consumption', 'Supplier',
       'Base Unit of Measure', 'Quantity', 'Material Doc. Year',
       'Debit/Credit ind', 'User name', 'Trans./Event Type', 'Currency',
       'Material Type', 'Vendor Code'],
      dtype='object')

In [79]:
df['Consumption'].notna().sum()

np.int64(3149)

In [71]:
pd.to_datetime(df['Entry Date'], format="%d-%m-%Y")
pd.to_datetime(df['Document Date'], format="%d-%m-%Y")
pd.to_datetime(df['Entry Date'], format="%d-%m-%Y")

0       2021-02-03
1       2021-02-02
2       2021-01-01
3       2020-12-31
4       2021-02-11
           ...    
61562   2024-03-29
61563   2024-03-28
61564   2024-03-22
61565   2024-03-29
61566   2024-03-29
Name: Entry Date, Length: 61226, dtype: datetime64[ns]

In [63]:
df.size

2081684